# PyESAPI: 10x Research
*By Michael.Folkerts@varian.com*

In [2]:
import pyesapi
import atexit
app = pyesapi.CustomScriptExecutable.CreateApplication('python_demo')
atexit.register(app.Dispose);

In [3]:
patient = app.OpenPatientById('TestUpperBody')
plan = patient.CoursesLot('C1').PlanSetupsLot('IMRT')
structures = plan.StructureSet.StructuresLot()

AttributeError: 'NoneType' object has no attribute 'CoursesLot'

In [4]:
patient